In [ ]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=b28f20438907dec9a5bb9b0c7436dbe355599f3549c9fc708aed1ba0ebf682dd
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
pip install nltk


In [ ]:
!pip install matplotlib


In [ ]:
import wikipedia
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import gensim.downloader as api
import numpy as np
documents=[
    "Artifitial Inteligence",
    "Neural Networks"
    "Deep Learning",
]
data=""

for title in documents:
  document_content = wikipedia.summary(title , sentences=200)
  data +=document_content

In [ ]:
print(data)

Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.
AI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often 

In [ ]:
print(len(data))

3771


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Get the list of English stopwords and punctuation
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

# Tokenize the text and remove stopwords and punctuation
tokens = word_tokenize(data)
data_words = [word.lower() for word in tokens if word.lower() not in stop_words and word.lower() not in punctuation]


In [ ]:
print(len(data_words))

375


In [ ]:
import numpy as np

word_vectors = api.load("glove-wiki-gigaword-100")
embedding_dim = 100

# Create a dictionary to store embeddings, where keys are words and values are embeddings
embeddings_dict = {}

for word in data_words:
    if word in word_vectors:
        embeddings_dict[word] = word_vectors[word]
    else:
        # Handle the case when the word is not in the vocabulary by generating a random embedding
        embeddings_dict[word] = np.random.rand(embedding_dim)

# Example of embeddings_dict: {'word1': embedding_vector1, 'word2': embedding_vector2, ...}


[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
print("Number of words in the dictionary:", len(embeddings_dict))# Convert embeddings dictionary values to numpy array
X = np.array(list(embeddings_dict.values()))

# Example of X shape
print("Shape of X:", X.shape)


Number of words in the dictionary: 255
Shape of X: (255, 100)


In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Assuming X_train contains a single sequence of word embeddings
X_train = X_train.reshape(1, X_train.shape[0], X_train.shape[1])
X_test = X_test.reshape(1, X_test.shape[0], X_test.shape[1])


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

if X_train.size == 0 or np.all(X_train == 0):
    print("Training data is empty. Cannot train the model.")
else:
    # Create labels by shifting X_train by one step
    y_train = np.roll(X_train, -1, axis=1)
    # The last element in y_train will be the same as the first element in X_train, so we remove it
    y_train[:, -1, :] = 0

    # Assuming X_train contains a single sequence of word embeddings
    number_of_words = X_train.shape[1]  # Number of words in the sequence
    embedding_dim = X_train.shape[2]  # Embedding dimension

    # Define RNN model
    model = Sequential([
        SimpleRNN(units=64, return_sequences=True, input_shape=(None, embedding_dim)),
        Dense(embedding_dim)  # Output layer with the same dimension as input (to predict the next word embedding)
    ])

    model.compile(optimizer='adam', loss='mae')  # Use MAE loss for predicting embeddings

    model.fit(X_train, y_train, epochs=10, batch_size=4)

model.summary()


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.5821
Epoch 2/10
1/1 [==============================] - 0s 223ms/step - loss: 0.5700
Epoch 3/10
1/1 [==============================] - 0s 313ms/step - loss: 0.5597
Epoch 4/10
1/1 [==============================] - 0s 294ms/step - loss: 0.5504
Epoch 5/10
1/1 [==============================] - 0s 269ms/step - loss: 0.5418
Epoch 6/10
1/1 [==============================] - 0s 306ms/step - loss: 0.5336
Epoch 7/10
1/1 [==============================] - 0s 290ms/step - loss: 0.5256
Epoch 8/10
1/1 [==============================] - 0s 123ms/step - loss: 0.5180
Epoch 9/10
1/1 [==============================] - 0s 135ms/step - loss: 0.5107
Epoch 10/10
1/1 [==============================] - 0s 130ms/step - loss: 0.5035
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, None, 64)          10560

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Function to save model summary as PDF
def save_model_summary_to_pdf(model, architecture_name):
    # Save model summary to a string
    summary_str = []
    model.summary(print_fn=lambda x: summary_str.append(x))
    summary_str = "\n".join(summary_str)

    # Create a plot for model summary
    plt.figure(figsize=(12, 6))
    plt.text(0.1, 0.5, summary_str, {'fontsize': 10}, fontfamily='monospace')
    plt.axis('off')

    # Save the plot as PDF
    pdf_file_path = f'/content/drive/MyDrive/{architecture_name}_model_summary.pdf'
    plt.savefig(pdf_file_path, bbox_inches='tight')
    plt.close()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Save model summary to PDF for the first architecture
save_model_summary_to_pdf(model, 'word_level_RNN')


In [ ]:
X_test.shape

(1, 51, 100)

In [ ]:
# Reshape X_test and X_test_labels to match the expected input shape

# Prepare labels for the test set
X_test_labels = np.roll(X_test, -1, axis=1)
X_test_labels[:, -1, :] = 0



In [ ]:
print(X_train.shape)
print(y_train.shape)

(1, 204, 100)
(1, 204, 100)


In [ ]:
loss = model.evaluate(X_test, X_test_labels)  # Assuming reconstruction loss
print("Test loss:", loss)

1/1 [==============================] - 0s 369ms/step - loss: 0.5242
Test loss: 0.5241793990135193


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def predict_next_word(word, model, embeddings_dict):
    # Get the embedding of the input word from the embeddings dictionary
    word_embedding = embeddings_dict[word]

    # Reshape the word embedding to match the input shape of the model
    word_embedding = np.expand_dims(word_embedding, axis=0)  # Add batch dimension
    word_embedding = np.expand_dims(word_embedding, axis=0)  # Add sequence dimension

    # Predict the embedding of the next word
    next_word_embedding = model.predict(word_embedding)

    # Remove the batch and sequence dimensions
    next_word_embedding = np.squeeze(next_word_embedding, axis=0)

    # Convert embeddings_dict values to a 2D numpy array
    embeddings_array = np.stack(list(embeddings_dict.values()))

    # Calculate cosine similarity between the predicted embedding and embeddings in the embeddings dictionary
    similarities = cosine_similarity(next_word_embedding, embeddings_array)

    # Find the index of the word with the highest similarity
    closest_word_index = np.argmax(similarities)

    # Get the word corresponding to the index
    closest_word = list(embeddings_dict.keys())[closest_word_index]

    return closest_word

# Example usage
seed_word = "intelligence"
predicted_word = predict_next_word(seed_word, model, embeddings_dict)
print("Predicted next word:", predicted_word)


1/1 [==============================] - 0s 284ms/step
Predicted next word: modern


In [ ]:
# Example usage
seed_word = "intelligence"
predicted_word = predict_next_word(seed_word, model, embeddings_dict)
print("Predicted next word:", predicted_word)

1/1 [==============================] - 0s 31ms/step
Predicted next word: modern


##* perform prediction based character*#

In [ ]:
def tokenize_characters(text):
    return [char for char in text if char.strip() and ord(char) not in [44, 39]]  # 44 is the Unicode code point for ',' and 39 is for "'"

data_characters = tokenize_characters(data.lower())

char_to_index = {char: i for i, char in enumerate(sorted(set(data_characters)))}

# Create the reverse mapping from index to characters
index_to_char = {i: char for char, i in char_to_index.items()}

# Number of unique characters
num_unique_chars = len(char_to_index)

# Define character embedding dimension
embedding_dim = 100

# Generate random embeddings for characters
embeddings_dict = {}
for char, index in char_to_index.items():
    embeddings_dict[char] = np.random.rand(embedding_dim)


In [ ]:
print(data_characters)

['a', 'r', 't', 'i', 'f', 'i', 'c', 'i', 'a', 'l', 'i', 'n', 't', 'e', 'l', 'l', 'i', 'g', 'e', 'n', 'c', 'e', '(', 'a', 'i', ')', 'i', 'n', 'i', 't', 's', 'b', 'r', 'o', 'a', 'd', 'e', 's', 't', 's', 'e', 'n', 's', 'e', 'i', 's', 'i', 'n', 't', 'e', 'l', 'l', 'i', 'g', 'e', 'n', 'c', 'e', 'e', 'x', 'h', 'i', 'b', 'i', 't', 'e', 'd', 'b', 'y', 'm', 'a', 'c', 'h', 'i', 'n', 'e', 's', 'p', 'a', 'r', 't', 'i', 'c', 'u', 'l', 'a', 'r', 'l', 'y', 'c', 'o', 'm', 'p', 'u', 't', 'e', 'r', 's', 'y', 's', 't', 'e', 'm', 's', '.', 'i', 't', 'i', 's', 'a', 'f', 'i', 'e', 'l', 'd', 'o', 'f', 'r', 'e', 's', 'e', 'a', 'r', 'c', 'h', 'i', 'n', 'c', 'o', 'm', 'p', 'u', 't', 'e', 'r', 's', 'c', 'i', 'e', 'n', 'c', 'e', 't', 'h', 'a', 't', 'd', 'e', 'v', 'e', 'l', 'o', 'p', 's', 'a', 'n', 'd', 's', 't', 'u', 'd', 'i', 'e', 's', 'm', 'e', 't', 'h', 'o', 'd', 's', 'a', 'n', 'd', 's', 'o', 'f', 't', 'w', 'a', 'r', 'e', 't', 'h', 'a', 't', 'e', 'n', 'a', 'b', 'l', 'e', 'm', 'a', 'c', 'h', 'i', 'n', 'e', 's',

In [ ]:
X = np.array(list(embeddings_dict.values()))


In [ ]:
# Define RNN model for character-level prediction
model = Sequential([
    SimpleRNN(units=64, input_shape=(1, embedding_dim), return_sequences=True),
    Dense(num_unique_chars, activation='softmax')  # Output layer predicts the next character
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Prepare input and output data for training
X = np.array([np.expand_dims(embeddings_dict[char], axis=0) for char in data_characters[:-1]])
y = np.array([char_to_index[char] for char in data_characters[1:]])  # Labels are indices of next characters

# Split the data into training and test sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train2, y_train2, epochs=10, batch_size=32)




Epoch 1/10
80/80 [==============================] - 3s 5ms/step - loss: 3.1188
Epoch 2/10
80/80 [==============================] - 0s 4ms/step - loss: 2.8852
Epoch 3/10
80/80 [==============================] - 0s 5ms/step - loss: 2.8122
Epoch 4/10
80/80 [==============================] - 0s 4ms/step - loss: 2.7519
Epoch 5/10
80/80 [==============================] - 0s 4ms/step - loss: 2.7125
Epoch 6/10
80/80 [==============================] - 0s 5ms/step - loss: 2.6783
Epoch 7/10
80/80 [==============================] - 0s 5ms/step - loss: 2.6550
Epoch 8/10
80/80 [==============================] - 0s 4ms/step - loss: 2.6375
Epoch 9/10
80/80 [==============================] - 0s 4ms/step - loss: 2.6184
Epoch 10/10
80/80 [==============================] - 0s 4ms/step - loss: 2.6086


In [ ]:
# Evaluate the model on the test data
loss = model.evaluate(X_test2, y_test2)
print("Test loss:", loss)

20/20 [==============================] - 0s 3ms/step - loss: 2.7006
Test loss: 2.70061993598938


In [ ]:
def predict_next_character(char, model, embeddings_dict):
    char_embedding = embeddings_dict[char]
    char_embedding = np.expand_dims(char_embedding, axis=0)  # Add batch dimension
    char_embedding = np.expand_dims(char_embedding, axis=0)  # Add sequence dimension

    next_char_embedding = model.predict(char_embedding)
    next_char_index = np.argmax(next_char_embedding)

    return index_to_char[next_char_index]

# Example usage
seed_character = "i"
predicted_character = predict_next_character(seed_character, model, embeddings_dict)
print("Predicted next character:", predicted_character)

1/1 [==============================] - 0s 190ms/step
Predicted next character: n


#* The second architecture


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

if X_train.size == 0 or np.all(X_train == 0):
    print("Training data is empty. Cannot train the model.")
else:
    # Create labels by shifting X_train by one step
    y_train = np.roll(X_train, -1, axis=1)
    # The last element in y_train will be the same as the first element in X_train, so we remove it
    y_train[:, -1, :] = 0

    # Assuming X_train contains a single sequence of word embeddings
    number_of_words = X_train.shape[1]  # Number of words in the sequence
    embedding_dim = X_train.shape[2]  # Embedding dimension

    # Define RNN model with an additional layer
    model = Sequential([
        SimpleRNN(units=64, return_sequences=True, input_shape=(None, embedding_dim)),
        SimpleRNN(units=64, return_sequences=True),  # Additional SimpleRNN layer
        Dense(embedding_dim)  # Output layer with the same dimension as input (to predict the next word embedding)
    ])

    model.compile(optimizer='adam', loss='mae')  # Use MAE loss for predicting embeddings

    model.fit(X_train, y_train, epochs=10, batch_size=4)

model.summary()


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.5913
Epoch 2/10
1/1 [==============================] - 0s 298ms/step - loss: 0.5688
Epoch 3/10
1/1 [==============================] - 0s 322ms/step - loss: 0.5542
Epoch 4/10
1/1 [==============================] - 0s 293ms/step - loss: 0.5430
Epoch 5/10
1/1 [==============================] - 0s 317ms/step - loss: 0.5327
Epoch 6/10
1/1 [==============================] - 0s 310ms/step - loss: 0.5224
Epoch 7/10
1/1 [==============================] - 0s 306ms/step - loss: 0.5117
Epoch 8/10
1/1 [==============================] - 0s 397ms/step - loss: 0.5010
Epoch 9/10
1/1 [==============================] - 1s 690ms/step - loss: 0.4903
Epoch 10/10
1/1 [==============================] - 1s 681ms/step - loss: 0.4800
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, None, 64)          105

In [ ]:
# Save model summary to PDF for the second architecture
save_model_summary_to_pdf(model, 'simpleRNN')


In [ ]:
# Reshape X_test and X_test_labels to match the expected input shape

# Prepare labels for the test set
X_test_labels = np.roll(X_test, -1, axis=1)
X_test_labels[:, -1, :] = 0



In [ ]:
loss = model.evaluate(X_test, X_test_labels)  # Assuming reconstruction loss
print("Test loss:", loss)

1/1 [==============================] - 0s 460ms/step - loss: 0.4911
Test loss: 0.49112769961357117


#Prediction  Based Character

In [ ]:
from tensorflow.keras.layers import Dropout

# Define RNN model for character-level prediction with an additional layer
model = Sequential([
    SimpleRNN(units=64, input_shape=(1, embedding_dim), return_sequences=True),
    SimpleRNN(units=64, return_sequences=True),  # Additional SimpleRNN layer
    Dropout(0.2),  # Dropout layer to prevent overfitting
    Dense(num_unique_chars, activation='softmax')  # Output layer predicts the next character
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Prepare input and output data for training
X = np.array([np.expand_dims(embeddings_dict[char], axis=0) for char in data_characters[:-1]])
y = np.array([char_to_index[char] for char in data_characters[1:]])  # Labels are indices of next characters

# Split the data into training and test sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train2, y_train2, epochs=10, batch_size=32)


Epoch 1/10
80/80 [==============================] - 4s 7ms/step - loss: 3.1362
Epoch 2/10
80/80 [==============================] - 1s 7ms/step - loss: 2.9084
Epoch 3/10
80/80 [==============================] - 1s 7ms/step - loss: 2.8150
Epoch 4/10
80/80 [==============================] - 1s 7ms/step - loss: 2.7465
Epoch 5/10
80/80 [==============================] - 1s 7ms/step - loss: 2.7117
Epoch 6/10
80/80 [==============================] - 1s 7ms/step - loss: 2.6875
Epoch 7/10
80/80 [==============================] - 1s 7ms/step - loss: 2.6728
Epoch 8/10
80/80 [==============================] - 1s 6ms/step - loss: 2.6564
Epoch 9/10
80/80 [==============================] - 1s 7ms/step - loss: 2.6589
Epoch 10/10
80/80 [==============================] - 1s 6ms/step - loss: 2.6226


In [ ]:
# Evaluate the model on the test data
loss = model.evaluate(X_test2, y_test2)
print("Test loss:", loss)

20/20 [==============================] - 1s 5ms/step - loss: 2.6899
Test loss: 2.6899356842041016


In [ ]:
# Example usage for character-based text generation
seed_character = "a"  # Starting character
iterations = 10

# Initialize the seed sequence with the starting characters
seed_chars = list("artificial intelligenc")  # Adjust as needed for your seed sequence

# Generate text for 10 iterations
for _ in range(iterations):
    predicted_character = predict_next_character(seed_chars[-1], model, embeddings_dict)
    if predicted_character:
        seed_chars.append(predicted_character)
        print(predicted_character, end='')
print()


1/1 [==============================] - 0s 24ms/step
e
